# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
! mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json


2024-02-12T12:32:50.152-0500	connected to: mongodb://localhost/
2024-02-12T12:32:50.155-0500	dropping: uk_food.establishments
2024-02-12T12:32:53.079-0500	39779 document(s) imported successfully. 0 document(s) failed to import.


In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
database_names = mongo.list_database_names()
database_names

['admin', 'classDB', 'config', 'epa', 'local', 'petsitly_marketing', 'uk_food']

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
collection_names = db.list_collection_names()
collection_names

['establishments']

In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())


{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65ca564212ffe2d0c7567670'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
 "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True}

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [11]:
# Check that the new restaurant was inserted
pprint(db.establishments.find_one({"BusinessName":"Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65ca5645858f8a3599b991bb'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

# Define the query to find the BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Define projection to return only BusinessTypeID and BusinessType fields
fields = {"BusinessTypeID": 1, "BusinessType": 1}
business_type_info = establishments.find_one(query, projection=fields)
pprint(business_type_info)


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65ca564212ffe2d0c7567670')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
# Extract the BusinessTypeID from the result
business_type_id = business_type_info["BusinessTypeID"]
# Update the new restaurant document with the correct BusinessTypeID
update_query = {"BusinessName": "Penang Flavours"}
new_values = {"$set": {"BusinessTypeID": business_type_id}}

# Perform the update operation
update_result = establishments.update_one(update_query, new_values)
pprint(update_result)

In [14]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(updated_restaurant)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65ca5645858f8a3599b991bb'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
# Count how many documents contain the Dover Local Authority
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
pprint(dover_count)

994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
pprint(delete_result.deleted_count)

994


In [17]:
# Check if any remaining documents include Dover
dover_recount= establishments.count_documents({"LocalAuthorityName": "Dover"})
pprint(dover_recount)

0


In [18]:
# Check that other documents remain with 'find_one'
remaining_count = establishments.count_documents({})
pprint(remaining_count)


38786


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
# import Decimal type
from bson.decimal128 import Decimal128
from decimal import Decimal

# Define the update query to convert latitude and longitude fields to decimal numbers
update_query = [
    {
        "$set": {
            "geocode.latitude": {"$convert": {"input": "$geocode.latitude", "to": "decimal"}},
            "geocode.longitude": {"$convert": {"input": "$geocode.longitude", "to": "decimal"}}
        }
    }
]

# Update the documents in the collection
update_result = establishments.update_many({}, update_query)

# Print the number of documents modified
pprint(update_result.modified_count)


38786


Use `update_many` to convert `RatingValue` to integer numbers.

In [23]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [24]:
# Change the data type from String to Integer for RatingValue
update_query_r = [
    {
        "$set": {
            "RatingValue": {"$toInt": "$RatingValue"}
        }
    }
]

# Update the documents in the collection
update_result_r = establishments.update_many({}, update_query_r)
pprint(update_result_r.modified_count)


34695


In [28]:
# Check that the coordinates and rating value are now numbers
# Find and print a sample document after the conversion
sample_document = establishments.find_one()

# Extract the latitude, longitude, and rating value fields
latitude = sample_document['geocode']['latitude']
longitude = sample_document['geocode']['longitude']
rating_value = sample_document['RatingValue']

# Print the data types of latitude, longitude, and rating value
print(type(latitude))
print(type(longitude))
print(type(rating_value))

<class 'bson.decimal128.Decimal128'>
<class 'bson.decimal128.Decimal128'>
<class 'int'>
